# RoboSaw Motor Specification Requirements Calculations

## Usage Notes
1. If using stepper motor without feedback, real-world torque must not exceed 40% to 50% of maximum pull-out torque ([trinamic](https://www.trinamic.com/technology/motor-control-technology/dcstep/))
2. Generally the stepper motor should not exceed 600 rpm if driven without feedback ([omc-stepperonline](https://www.omc-stepperonline.com/support-stepper-motor/what-is-the-maximum-speed-highest-frequency-of-the-motor))

## Intake Stepper Motor Requirements

In [1]:
import numpy as np
import pandas as pd
import pint
import matplotlib.pyplot as plt
from IPython.display import display
import sympy as sp

In [2]:
# initial variables
ureg = pint.UnitRegistry()

cf_static_wood = 0.6
wt_wood = {'2_4': 1.3, '2_6' : 2.0, '4_4' : 4.8} # lb/ft
wt_wood = {key: val * ureg.lbf / ureg.ft for (key, val) in wt_wood.items()}
intake_wheel_diam = 2.25 * ureg.inch # in
max_length = 8 * ureg.ft # ft
fos = 1.5 # factor of safety

In [3]:
def force_frict(coef, wt):
    return coef * wt

def torque_frict(diam, force):
    return diam / 2 * force

In [4]:
force_max = force_frict(cf_static_wood, max_length * wt_wood[max(wt_wood, key=wt_wood.get)])
torque_max = torque_frict(intake_wheel_diam, force_max)

print(f"The maximum force is {force_max:.1f}")
print(f"The maximum torque with an intake wheel with diameter of {intake_wheel_diam} is {torque_max:.1f}\n")

print(f"Look for stepper motor with holding torque of {torque_max * fos:.1f}, {torque_max.to('ozf*inch') * fos:.1f}")

The maximum force is 23.0 force_pound
The maximum torque with an intake wheel with diameter of 2.25 inch is 25.9 force_pound * inch

Look for stepper motor with holding torque of 38.9 force_pound * inch, 622.1 force_ounce * inch


In [5]:
# must remove units for pandas compatibility

length = np.arange(1, max_length.magnitude, 1) * ureg.ft # ft

intake_recs = pd.DataFrame({'length' : length.magnitude})
for key, val in wt_wood.items():
    intake_recs["force_" + key] = force_frict(cf_static_wood, val.magnitude * intake_recs.length)
    intake_recs["torque_" + key] = torque_frict(intake_wheel_diam.magnitude, intake_recs["force_" + key])
display(intake_recs)

,length,force_2_4,torque_2_4,force_2_6,torque_2_6,force_4_4,torque_4_4
0,1,0.78,0.8775,1.2,1.35,2.88,3.24
1,2,1.56,1.7550,2.4,2.70,5.76,6.48
2,3,2.34,2.6325,3.6,4.05,8.64,9.72
3,4,3.12,3.5100,4.8,5.40,11.52,12.96
4,5,3.90,4.3875,6.0,6.75,14.40,16.20
5,6,4.68,5.2650,7.2,8.10,17.28,19.44
6,7,5.46,6.1425,8.4,9.45,20.16,22.68


In [6]:
dist_normal = 36 * ureg.inch
dist_force_app = 2 * ureg.inch
length_wood = 8 * ureg.ft
wt_wood_tot = wt_wood['4_4'] * length_wood
print(wt_wood_tot)
print(length_wood)

force_app = (wt_wood_tot * (length_wood / 2 - dist_normal)) / (dist_normal - dist_force_app)
print(force_app.to('lbf'))

38.4 force_pound
8 foot
13.552941176470588 force_pound


In [7]:
def torque(wheel_radius, force):
    return wheel_radius * force

def linear_speed(wheel_radius, speed_motor, gr):
    return 2* np.pi * wheel_radius * speed_motor / gr

intake_force = 10 * ureg.lbf
wheel_radius = 2.25 / 2 * ureg.inch
speed_motor = 84 * ureg.rpm
gear_ratio = 1

intake_torque = torque(wheel_radius, intake_force)
print(intake_torque.to('ozf in'))

wood_speed = linear_speed(wheel_radius, speed_motor, gear_ratio)
print(wood_speed.to('ft / s'))

180.00000000000003 force_ounce * inch
5.1815423105719125 foot / second


## Turn Table Stepper Motor Requirements

In [8]:
def torque_pinion(force, pd_pinion):
    return force * pd_pinion / 2

def speed_table(vel_motor, gear_ratio, pd_pinion, motor_loc_radius):
    return (vel_motor / gear_ratio * pd_pinion / 2) / motor_loc_radius

In [9]:
pd_pinion = 1.5 * ureg.inch # in
force_meas = 6 * ureg.lbf # lbf
vel_motor = 600 * ureg.rpm # rpm
gear_ratio_stepper = 14
force_meas_radius = 197.5 * ureg.millimeter # mm
motor_loc_radius = 109.5 * ureg.millimeter # mm

force_rack = (force_meas_radius / motor_loc_radius) * force_meas

torque_pinion_motor = torque_pinion(force_rack, pd_pinion)
print(f"The maximum torque required to move the rotary table is {torque_pinion_motor.to('lbf * in') * fos:.1f}, {torque_pinion_motor.to('ozf * in') * fos:.1f}")
max_speed_table = speed_table(vel_motor, gear_ratio_stepper, pd_pinion, motor_loc_radius)
print(f"The maximum speed of the turn_table is {max_speed_table.to('deg/s'):.1f}")

The maximum torque required to move the rotary table is 12.2 force_pound * inch, 194.8 force_ounce * inch
The maximum speed of the turn_table is 44.7 degree / second


## Saw Blade Fence Motor Requirements

In [10]:
min_wood_width = 3.5 * ureg.inch
miter_offset = 197.5 * ureg.millimeter
fence_offset = 20 * ureg.millimeter

max_extension = miter_offset - min_wood_width + fence_offset
clamp_angle = np.arcsin(fence_offset/max_extension) * ureg.radian
clamp_force = 10 * ureg.lbf
torque_fence = clamp_force * max_extension * np.cos(clamp_angle)
print(max_extension.to('inch'))
print(clamp_angle.to('degree'))
print(torque_fence.to('ozf * in'))

5.062992125984254 inch
8.947013514869179 degree
800.2221775926157 force_ounce * inch


## Saw Blade Linear Actuator Motor Requirements